# 1. Setup

Seção para realizar a prepração do ambiente de desenvolvimento, realizando a conexão com o Google Drive e importando as bibliotecas necessárias e importação das bases fornecidas.


In [472]:
# Montagem do drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [473]:
# Importar bibliotecas necessárias
import pandas as pd
import re

## Carregamento dos dados

Para realizar o pré processamento foi utilizada 3 bases de dados que possuem semelhanças:

**Base 1**: Base original apenas com conversas e intenções (descartada, muito antiga e entra em conflito com as outras)

**Base 2**: Base secundária com conversas e mais de uma interação

**Base 3**: Base terciária com um dicionário para as palavras em japonês

In [474]:
# Caminho das bases
base2_path = '/content/drive/Shareddrives/Grupo_05/base2_v1_20240904.xlsx'
base3_path = '/content/drive/Shareddrives/Grupo_05/base3_v1_20240904.xlsx'

### Tratamento Q&A

Retirar linhas duplicadas, filtrar colunas principais e exploração da base.

In [475]:
# Importação das bases 2 e 3 Q&A

base2_QA = pd.read_excel(base2_path, sheet_name='Q&A')
base3_QA = pd.read_excel(base3_path, sheet_name='Q&A')

In [476]:
# Print all values that present in column "Intencao" that are present in base2_QA but not base3_QA, and vice-versa
print(set(base2_QA['Intencao']) - set(base3_QA['Intencao']))
print(set(base3_QA['Intencao']) - set(base2_QA['Intencao']))

{'Envio via Deposit Code', 'Como depositar ', 'Problemas de remessa', 'Como fazer remessa', 'Atualizacao de dados cadastrais', 'Registro/Atualizacao de Documento', 'Tempo para depositos refletirem no saldo'}
{'Problemas/Duvidas sobre deposito', 'Problemas/Duvidas sobre remessas', 'Problemas/Duvidas de atualizacao de dados cadastrais', 'Dificuldades com utilizacao do App'}


In [477]:
# Junção das bases
baseGeral_QA = pd.merge(base2_QA, base3_QA, how='outer')

In [478]:
# Remove uma coluna sem nome que contem somente IDs
baseGeral_QA = baseGeral_QA[['Intencao', 'Pergunta', 'Resposta\n']]

In [479]:
# Renomeia o erro de digitação em 'Resposta'
baseGeral_QA.rename(columns={'Resposta\n':'Resposta'}, inplace=True)

In [480]:
# Retirar Q&As que estão duplicadas
baseGeral_QA.drop_duplicates(inplace=True)

In [481]:
# Reinicia os indexes
baseGeral_QA.reset_index(drop=True, inplace=True)

In [482]:
# Informações para averiguar se os processos foram executados com sucesso
print(baseGeral_QA.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 792 entries, 0 to 791
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Intencao  786 non-null    object
 1   Pergunta  791 non-null    object
 2   Resposta  781 non-null    object
dtypes: object(3)
memory usage: 18.7+ KB
None


In [483]:
baseGeral_QA.head()

,Intencao,Pergunta,Resposta
0,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aquele caixa do family mart verde eh 24 horas?e se eu posso transferir a qualquer hora?obrigado,"Bom dia! Sim, o sr pode utilizar o cartão de depósitos nesta máquinas 24 horas exclusivas do JP Bank."
1,Como fazer remessa,Como enviar dinheiro do Japão?,NaN
2,Tempo de remessa,Quanto tempo levará para o beneficiário receber o dinheiro?,"Via de regra, as remessas serão pagas via PIX dentro de algumas horas, se todos os dados estiverem corretos."
3,"Pedido de envio via metodo ""ByPhone""","Boa tarde\nAcabei de fazer a transferência de 22+23, Total de 45Yenes\n",iremos processar a sua solicitacao.\nMuito obrigada e otima tarde.
4,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB RRR obrigado\nE de 23yenes para o AAAA MMMM VVVVV caixa econômica federal\nObrigado,iremos processar a sua solicitacao.\nMuito obrigada e otima tarde.


### Merge das Bases

In [496]:
baseGeral = baseGeral_QA

baseGeral = baseGeral[['Intencao', 'Pergunta', 'Resposta']]

In [497]:
baseGeral = baseGeral.replace('', pd.NA)
baseGeral = baseGeral.dropna()

In [498]:
baseGeral.head()

,Intencao,Pergunta,Resposta
0,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aquele caixa do family mart verde eh 24 horas?e se eu posso transferir a qualquer hora?obrigado,"Bom dia! Sim, o sr pode utilizar o cartão de depósitos nesta máquinas 24 horas exclusivas do JP Bank."
2,Tempo de remessa,Quanto tempo levará para o beneficiário receber o dinheiro?,"Via de regra, as remessas serão pagas via PIX dentro de algumas horas, se todos os dados estiverem corretos."
3,"Pedido de envio via metodo ""ByPhone""","Boa tarde\nAcabei de fazer a transferência de 22+23, Total de 45Yenes\n",iremos processar a sua solicitacao.\nMuito obrigada e otima tarde.
4,"Pedido de envio via metodo ""ByPhone""",Poderia fazer a remessa de 22yenes para o BBB RRR obrigado\nE de 23yenes para o AAAA MMMM VVVVV caixa econômica federal\nObrigado,iremos processar a sua solicitacao.\nMuito obrigada e otima tarde.
5,"Pedido de envio via metodo ""ByPhone""",Boa tarde\nAcabei de fazer uma transferência para conta de vocês no valor de 50.700 a ser descontado a taxa de vocês\n,Bom dia! Processaremos conforme solicitado. Obrigado!


In [499]:
baseGeral.info()

<class 'pandas.core.frame.DataFrame'>
Index: 776 entries, 0 to 786
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Intencao  776 non-null    object
 1   Pergunta  776 non-null    object
 2   Resposta  776 non-null    object
dtypes: object(3)
memory usage: 24.2+ KB


# 2. Pipeline

**Etapa importante para lidar com os textos do dataframe futuramente.**

**Nesta etapa, o processo de pré-processamento de dados foi separado nas seguintes etapas a seguir:**

Limpeza dos dados do dataframe: remover os espaços em excesso e converter o texto em uma frase de uma linha

## Normalização e remoção de dados que não serão utilizados

In [500]:
# Substituir quebras de linha por espaços
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\n', ' ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\n', ' ', str(x)))

# Remover espaços extras do dataframe
baseGeral['Pergunta'] = baseGeral['Pergunta'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))
baseGeral['Resposta'] = baseGeral['Resposta'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))

# Remove rows with empty strings in any column
baseGeral = baseGeral.replace('', pd.NA)
baseGeral = baseGeral.dropna()

# Remove rows where any column contains only newline characters or whitespace
baseGeral = baseGeral[~baseGeral.applymap(lambda x: isinstance(x, str) and re.match(r'^\s*$', x)).any(axis=1)]

<ipython-input-500-a1ccc49d3d2d>:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  baseGeral = baseGeral[~baseGeral.applymap(lambda x: isinstance(x, str) and re.match(r'^\s*$', x)).any(axis=1)]


## Dicionário das abreviações encontradas na base de dados

Dicionário para conversão de abreviações para palavras por extenso: foi feita uma análise das principais abreviações que apareciam no dataframe e, com base nisso, criado um dicionário que é usado para substituir cada abreviação por sua palavra por extenso através de um loop que percorre as principais colunas que estamos usando;


In [501]:
# Criação de um dicionário para tratar abreviações do dataframe
abreviacoes = {
    'add': 'adicionar',
    'app': 'aplicativo',
    'pq': 'porque',
    'vc': 'você',
    'vcs': 'vocês',
    'sr': 'senhor',
    'sra': 'senhora',
    'srto': 'senhorito',
    'srta': 'senhorita',
    'qdo': 'quando',
    'qdo': 'quando',
    'qt': 'quanto',
    'qto': 'quanto',
    'pra': 'para',
    'q': 'que',
    'p': 'para',
    'pf': 'por favor',
    'yenes': 'ienes',
    'yens': 'ienes',
    'iens': 'ienes',
    'hj': 'hoje',
    'obg': 'obrigado',
    'td': 'tudo',
    'bm': 'bom',

    'usuario': 'usuário',
    'Usuario': 'Usuário',
    'voce': 'você',
    'Voce': 'Você',
    'nao': 'não',
    'Nao': 'Não',
    'ola': 'olá',
    'Ola': 'Olá'
}

In [502]:
# Substituindo as abreviações das palavras para extenso em toda a base de dados de uma vez
def substituir_abreviacoes(texto, abreviacoes):
    for abreviacao, palavra in abreviacoes.items():
        texto = texto.str.replace(r'\b' + abreviacao + r'\b', palavra, regex=True)
    return texto

# Aplicando a função para substituir abreviações nas colunas específicas
baseGeral['Pergunta'] = substituir_abreviacoes(baseGeral['Pergunta'], abreviacoes)
baseGeral['Resposta'] = substituir_abreviacoes(baseGeral['Resposta'], abreviacoes)

In [503]:
baseGeral.head(1)

,Intencao,Pergunta,Resposta
0,Como depositar,Boa dia.tudo bem?eu gostaria de saber sobre aquele caixa do family mart verde eh 24 horas?e se eu posso transferir a qualquer hora?obrigado,"Bom dia! Sim, o senhor pode utilizar o cartão de depósitos nesta máquinas 24 horas exclusivas do JP Bank."


In [504]:
import random

random_row = random.randint(0, len(baseGeral) - 1)
pd.set_option('display.max_colwidth', None)
print(baseGeral.iloc[[random_row]])

                    Intencao  \
59  Cadastro de beneficiario   

                                                                           Pergunta  \
59  Boa trde como faco para mudar da conta corrente para a poupanca do beneficiario   

                                                                                                                                                                                                                                                                                                                                                                             Resposta  
59  Boa tarde. No seu aplicativo Barstel Remit, click em Menu >> Beneficiarios >> Escolha o beneficiario (click) >> EDITAR >>Faca as correcoes/atualizacoes e click em Proximo (se não aparecer esse botao, click levente em qualquer lugar da sua tela e o botao ira aparecer) >> Va clicando no proximo ate apararecer SALVAR Qualquer outra dúvida estamos à disposição. Obrigado.  


In [505]:
# Apagar colunas não utilizadas e eliminar linhas com valores nulos
baseGeral = baseGeral.drop_duplicates(keep='first')
baseGeral.info()

<class 'pandas.core.frame.DataFrame'>
Index: 759 entries, 0 to 786
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Intencao  759 non-null    object
 1   Pergunta  759 non-null    object
 2   Resposta  759 non-null    object
dtypes: object(3)
memory usage: 23.7+ KB


In [506]:
# União de labels com propósitos similares, isto ajuda a categorizar melhor as intenções.
baseGeral.loc[baseGeral['Intencao'].str.contains('Como depositar', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Como depositar', case=False, na=False), 'Intencao'].replace('Como depositar', 'Problemas/Duvidas sobre deposito', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Como fazer remessa', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Como fazer remessa', case=False, na=False), 'Intencao'].replace('Como fazer remessa', 'Problemas/Duvidas sobre remessas', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Problemas de remessa', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Problemas de remessa', case=False, na=False), 'Intencao'].replace('Problemas de remessa', 'Problemas/Duvidas sobre remessas', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Problemas/Duvidas sobre deposito ', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Problemas/Duvidas sobre deposito ', case=False, na=False), 'Intencao'].replace('Problemas/Duvidas sobre deposito ', 'Problemas/Duvidas sobre deposito', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Atualizacao de dados cadastrais', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Atualizacao de dados cadastrais', case=False, na=False), 'Intencao'].replace('Atualizacao de dados cadastrais', 'Problemas/Duvidas de atualizacao de dados cadastrais', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Registro/Atualizacao de Documento', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Registro/Atualizacao de Documento', case=False, na=False), 'Intencao'].replace('Registro/Atualizacao de Documento', 'Problemas/Duvidas de atualizacao de dados cadastrais', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Tempo para depositos refletirem no saldo', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Tempo para depositos refletirem no saldo', case=False, na=False), 'Intencao'].replace('Tempo para depositos refletirem no saldo', 'Tempo de remessa', regex=True)
baseGeral.loc[baseGeral['Intencao'].str.contains('Envio via Deposit Code', case=False, na=False), 'Intencao'] = baseGeral.loc[baseGeral['Intencao'].str.contains('Envio via Deposit Code', case=False, na=False), 'Intencao'].replace('Envio via Deposit Code', 'Problemas/Duvidas sobre deposito', regex=True)

In [507]:
# Drop rows where the number of intentions in all dataset is less than 10
intentions_counts = baseGeral['Intencao'].value_counts()
intentions_to_drop = intentions_counts[intentions_counts < 10].index
baseGeral = baseGeral[~baseGeral['Intencao'].isin(intentions_to_drop)]

In [508]:
# Remove all rows where the value in Pergunta and Resposta are duplicated with other rows, keep at least one of them
baseGeral = baseGeral.drop_duplicates(subset=['Pergunta', 'Resposta'], keep='first')

In [509]:
baseGeral['Intencao'].value_counts()

,count
Intencao,
Confirmacao de cambio/taxas,118
"Pedido de envio via metodo ""ByPhone""",98
Problemas/Duvidas de atualizacao de dados cadastrais,79
Problemas/Duvidas sobre remessas,66
Como se inscrever,56
Problemas/Duvidas sobre deposito,45
Tempo de remessa,40
Solicitacao de cartao de remessas,35
Termos e condicoes do servico,32


## Baixar base tratada

In [511]:
# Salvar a base de dados tratada
baseGeral.to_excel('NLG - baseTratadaSprint5.xlsx')